In [1]:
W:t!pip install trl accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import transformers
import pandas as pd
import numpy as np
from datasets import load_dataset
from torch.utils.data import Dataset
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
from dataclasses import dataclass
import datasets

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from tqdm.notebook import tqdm

In [4]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead

In [5]:
tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

In [7]:
ppo_config = {
    "output_dir": "/content/drive/MyDrive/PPO",
    "do_train": True,
    "do_eval": True,
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 16,
    "per_device_eval_batch_size" : 1,
    "learning_rate": 5e-6,
    "num_train_epochs": 3,
    "logging_steps": 100,
    "save_steps": 100,
    "save_strategy": "steps",
    "eval_strategy": "no",
    "report_to": "tensorboard",
    "bf16": True,
    "remove_unused_columns": True,
    "logging_dir": "/content/drive/MyDrive/PPO/logs",
    "seed": 42,

    # PPO-specific
    "num_ppo_epochs": 4,
    "cliprange": 0.2,
    "cliprange_value": 0.2,
    "vf_coef": 0.1,
    "gamma": 1.0,
    "lam": 0.95,
    "kl_coef": 0.05,
}

In [6]:
generation_kwargs = {
    "max_new_tokens": 256,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id,
    "no_repeat_ngram_size": 4,
    "do_sample": True,
    "top_k": 40,
    "temperature": 0.7,
}

In [9]:
ppo_config = PPOConfig(**ppo_config) #mini_batch_size

In [11]:
ppo_config.response_length=256

In [11]:
from trl import PPOConfig
ppo_config = PPOConfig()
print(ppo_config.to_dict())

{'output_dir': 'trainer_output', 'overwrite_output_dir': False, 'do_train': False, 'do_eval': False, 'do_predict': False, 'eval_strategy': 'no', 'prediction_loss_only': False, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 8, 'per_gpu_train_batch_size': None, 'per_gpu_eval_batch_size': None, 'gradient_accumulation_steps': 1, 'eval_accumulation_steps': None, 'eval_delay': 0, 'torch_empty_cache_steps': None, 'learning_rate': 5e-05, 'weight_decay': 0.0, 'adam_beta1': 0.9, 'adam_beta2': 0.999, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 3.0, 'max_steps': -1, 'lr_scheduler_type': 'linear', 'lr_scheduler_kwargs': {}, 'warmup_ratio': 0.0, 'warmup_steps': 0, 'log_level': 'passive', 'log_level_replica': 'warning', 'log_on_each_node': True, 'logging_dir': 'trainer_output/runs/Jun01_07-11-02_63712dd7b2ad', 'logging_strategy': 'steps', 'logging_first_step': False, 'logging_steps': 500, 'logging_nan_inf_filter': True, 'save_strategy': 'steps', 'save_steps': 500

In [17]:
class GPTRewardModel(torch.nn.Module):
  base_model_prefix = "backbone"
  def __init__(self, model_path):
    super().__init__()
    self.backbone = AutoModel.from_pretrained(model_path)
    self.v_head = torch.nn.Linear(self.backbone.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask, **kwargs):
    out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
    hidden = out.last_hidden_state
    last_token_index = attention_mask.sum(dim=1) - 1
    batch_idx = torch.arange(input_ids.size(0), device=input_ids.device)
    pooled = hidden[batch_idx, last_token_index]
    logits = self.v_head(pooled)
    return logits.view(-1)

  def score(self, hidden_states, attention_mask=None, **kwargs):
    return self.v_head(hidden_states).squeeze(-1)

  @property
  def config(self):
    return self.backbone.config  # or self.backbone.model.config if needed

In [15]:
rm_model = GPTRewardModel(model_path="/content/drive/MyDrive/rm_backbone")
v_head_weights = torch.load("/content/drive/MyDrive/rm_backbone/v_head.bin", map_location="cpu")
rm_model.v_head.load_state_dict(v_head_weights)
rm_model.eval()
for param in rm_model.parameters():
    param.requires_grad = False

In [18]:
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

class CriticWithScore(AutoModelForCausalLMWithValueHead):
  def score(self, hidden_states):
    return self.v_head(hidden_states).squeeze(-1)

  def forward(self, *args, **kwargs):
    kwargs.setdefault("output_hidden_states", True)
    kwargs.setdefault("return_dict", True)

    # 1차 호출
    outputs = super().forward(*args, **kwargs)

    # hidden_states 없으면 수동으로 backbone에서 추출
    if not hasattr(outputs, 'hidden_states'):
      backbone = getattr(self, self.base_model_prefix)
      transformer_outputs = backbone(*args, **kwargs)
      hidden_states = transformer_outputs.hidden_states

      outputs = CausalLMOutputWithCrossAttentions(
          loss=outputs.loss if hasattr(outputs, "loss") else None,
          logits=outputs.logits,
          past_key_values=outputs.past_key_values if hasattr(outputs, "past_key_values") else None,
          hidden_states=hidden_states,
          attentions=outputs.attentions if hasattr(outputs, "attentions") else None,
          cross_attentions=None,
      )

    return outputs

In [19]:
model_name="/content/drive/MyDrive/merged_model"

actor_model = AutoModelForCausalLM.from_pretrained(model_name)
#critic_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
critic_model = CriticWithScore.from_pretrained(model_name)

ref_model = copy.deepcopy(actor_model)
ref_model.eval()
for param in ref_model.parameters():
    param.requires_grad = False

In [20]:
actor_model = copy.deepcopy(actor_model)

In [21]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config1 = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"],
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

actor_model = get_peft_model(actor_model, lora_config1)
actor_model.print_trainable_parameters()

trainable params: 8,110,080 || all params: 1,170,666,240 || trainable%: 0.6928


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [83]:
for name, param in actor_model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.shape}")

base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight: torch.Size([5760, 16])
base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight: torch.Size([1920, 16])
base_model.model.transformer.h.0.mlp.c_proj.lora_A.default.weight: torch.Size([16, 7680])
base_model.model.transformer.h.0.mlp.c_proj.lora_B.default.weight: torch.Size([1920, 16])
base_model.model.transformer.h.1.attn.c_attn.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.transformer.h.1.attn.c_attn.lora_B.default.weight: torch.Size([5760, 16])
base_model.model.transformer.h.1.attn.c_proj.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.transformer.h.1.attn.c_proj.lora_B.default.weight: torch.Size([1920, 16])
base_model.model.transformer.h.1.mlp.c_proj.lora_A.default.weight: torch.Size([16, 7680])
ba

In [22]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config2 = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"],
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

critic_model = get_peft_model(critic_model, lora_config2)
critic_model.print_trainable_parameters()

trainable params: 8,110,080 || all params: 1,170,668,161 || trainable%: 0.6928


In [15]:
for name, param in critic_model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.shape}")

base_model.model.pretrained_model.transformer.h.0.attn.c_attn.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.pretrained_model.transformer.h.0.attn.c_attn.lora_B.default.weight: torch.Size([5760, 16])
base_model.model.pretrained_model.transformer.h.0.attn.c_proj.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.pretrained_model.transformer.h.0.attn.c_proj.lora_B.default.weight: torch.Size([1920, 16])
base_model.model.pretrained_model.transformer.h.0.mlp.c_proj.lora_A.default.weight: torch.Size([16, 7680])
base_model.model.pretrained_model.transformer.h.0.mlp.c_proj.lora_B.default.weight: torch.Size([1920, 16])
base_model.model.pretrained_model.transformer.h.1.attn.c_attn.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.pretrained_model.transformer.h.1.attn.c_attn.lora_B.default.weight: torch.Size([5760, 16])
base_model.model.pretrained_model.transformer.h.1.attn.c_proj.lora_A.default.weight: torch.Size([16, 1920])
base_model.model.pretrained_mo

In [23]:
for name, param in critic_model.named_parameters():
    if "v_head" in name:
        param.requires_grad = True
        print(f"Trainable: {name}")

Trainable: base_model.model.v_head.summary.weight
Trainable: base_model.model.v_head.summary.bias


In [24]:
critic_model.base_model_prefix = "pretrained_model"

In [25]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
df = pd.read_parquet("hf://datasets/royboy0416/ko-alpaca/data/ko_alpaca_data.snappy.parquet")
dataset = datasets.Dataset.from_pandas(df)

def format_prompt(example):
    if example["input"]:
        prompt = f"{example['instruction']}\n{example['input']}"
    else:
        prompt = example["instruction"]
    return {
        "prompt": prompt,
        "response": example["output"]
    }

dataset = dataset.map(format_prompt)

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [10]:
class PPODataset(Dataset):
  def __init__(self, data, tokenizer, max_length=1024):
    self.data = data
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    prompt = self.data[idx]["prompt"]
    PROMPT_TEMPLATE = "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    prompt = PROMPT_TEMPLATE.format(prompt=prompt)

    tokenized = self.tokenizer(prompt, padding = 'longest', truncation=True, max_length=self.max_length, return_tensors="pt")

    return {
        "input_ids": tokenized["input_ids"].squeeze(0),
        #"attention_mask": tokenized["attention_mask"].squeeze(0),
    }

In [11]:
ppo_train=PPODataset(dataset, tokenizer)

In [30]:
clean_data = []
for i in range(len(ppo_train)):
    sample = ppo_train[i]
    clean_data.append({
        "input_ids": [int(x) for x in sample["input_ids"].tolist()],
    })

ppo_train_dataset = datasets.Dataset.from_list(clean_data)
ppo_train_dataset.set_format(type="torch")

In [31]:
split_dataset = ppo_train_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)
ppo_train = split_dataset["test"]
ppo_test = split_dataset['train']

In [34]:
print(actor_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(51200, 1920)
        (wpe): Embedding(1024, 1920)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=5760, nx=1920)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1920, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5760, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
   

In [39]:
actor_model.base_model.model.generation_config

GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 8,
  "pad_token_id": 8
}

In [40]:
from transformers import GenerationConfig

actor_model.generation_config = GenerationConfig(**generation_kwargs)
actor_model.base_model.generation_config = GenerationConfig(**generation_kwargs)
actor_model.base_model.model.generation_config = GenerationConfig(**generation_kwargs)

In [41]:
trainer = PPOTrainer(
    args=ppo_config,                      # PPOConfig instance
    processing_class=tokenizer,          # Tokenizer
    model=actor_model,                   # LoRA-wrapped actor model
    ref_model=ref_model,                 # Frozen reference model
    reward_model=rm_model,           # Reward model (can be a callable or model)
    train_dataset=ppo_train,     # Your training dataset (must support __len__ and __getitem__)
    value_model=critic_model,            # Critic model (with ValueHead)
    data_collator=data_collator,        # HuggingFace-style data collator
    eval_dataset=datasets.Dataset.from_list([ppo_train[0],ppo_train[1]])
    )

In [30]:
torch.cuda.empty_cache()

In [43]:
trainer.eval_dataset = None
trainer.eval_dataloader = None
trainer.generate_completions = lambda *args, **kwargs: []

In [69]:
ppo_config.num_sample_generations

10

In [44]:
trainer.train()

===training policy===


`generation_config` default values have been modified to match model-specific defaults: {'no_repeat_ngram_size': 4, 'pad_token_id': 3}. If this is not desired, please set these values explicitly.


Step,Training Loss


평가

In [1]:
%load_ext tensorboard

In [3]:
%tensorboard --logdir /content/drive/MyDrive/PPO/logs

<IPython.core.display.Javascript object>

1. policy loss와 value loss는 수렴하였다. 조심스러운 것이 value loss 자체는 reward regression 형태와 비슷하기 때문에 수렴하기 쉽고 따라서 advantage가 줄어듬에 따라 policy loss도 감소한 것이 아닐까 싶다. 따라서 단순 수렴만으로 policy 개선을 판단하기 어려우며 reward와 advantage 경향성을 확인해야한다.
2. KL을 보면 policy update가 학습초기에 크다가 이후에는 policy update가 sft model과 stable하게 수렴하는 것을 보여준다.
3. non-scroe reward는 rm의 reward 결과를 보여주는데 모델이 rm의 reward를 안정적으로 따라간 것으로 보인다. 다만 rm이 상대적 크기가 중요하더라도 reward가 음수 영역에서 나오는 것은 rm 자체의 평균 reward를 확인해 봐야 할 것같다. 필요시 reward normalization 이나 score shifting을 고려할 수 있다.
4. entropy 는 증가하는 경향을 보이는데 결정적인 explotiation 영역에서 탐색적인 exploration 영역으로 진행되었다. 엔트로피는 actor model이 좋은 policy 방향으로 학습되면서 감소하는 것이 일반적인데 rm 기준을 beam과 greedy 샘플의 비교를 넣은 것과 policy grad 대상이 actor_model의 sample generate 기준이었기 때문일 것 같다. 특히 n-gram repeat 방지는 모델에게 탐색을 더 강요한다.
5. entropy 증가의 다른 이유로는 rm sharpness의 부족이 우려된다. gold와 beam, greedy의 비교 정확도는 90 이상이 나왔지만 score 자체의 scale차이가 크지 않을 수 있다. 확인이 필요하다. rm dataset 품질을 올리거나 직접적으로 rm score scaling을 해서 reward의 차이와 영향을 키울 수 있다.
6. rlhf reward, score를 보면 actor_model이 reward를 잘 따라가고 있음을 확인할 수 있다. 다만 reward가 증가해도 entropy가 증가하는 것은 모델이 보상을 받기 위해 다양한 전략을 탐색하는 것으로 판단할 수 있다.
7. 추가적으로 rm을 마지막 유효토큰 기준으로 계산한 것(원본코드는 마지막 토큰 제외 평균)과 kl coeff, gae, ppo coeff 등 실험조건의 영향을 비교해 볼 수 있다.

In [7]:
from peft import PeftModel, PeftConfig

model_name="/content/drive/MyDrive/merged_model"

base_model = AutoModelForCausalLM.from_pretrained(model_name)
actor_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/PPO/checkpoint-466")

In [8]:
actor_model = actor_model.merge_and_unload()

actor_model.save_pretrained("/content/drive/MyDrive/PPO/PPO_result")

In [17]:
greedy_generation_args = dict(
    do_sample=False,
    max_new_tokens=256,
    eos_token_id=1,
    repetition_penalty=1.0,
    no_repeat_ngram_size=0,
    batch_size=16,
)

In [19]:
beam_generation_args = dict(
    num_beams=5,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=1,
    max_new_tokens=256,
    do_sample=True,
    top_k=40,
    early_stopping=True,
    temperature = 0.7
)

In [28]:
generator = pipeline('text-generation', model='/content/drive/MyDrive/merged_model', tokenizer=tokenizer)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [ dataset['prompt'][30], dataset['prompt'][500] ,dataset['prompt'][1100], dataset['prompt'][2000]]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **beam_generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
식물의 세포 호흡 과정을 설명하십시오.

### Response(응답):식물은 광합성을 통해 이산화탄소를 흡수하고 산소를 방출하여 에너지를 생성합니다.

### Instruction(명령어):
로미오와 줄리엣에 나오는 로미오의 성격을 묘사하십시오.

### Response(응답):로미오는 사랑스럽고 친절하며 로맨틱한 성격을 가지고 있습니다.

### Instruction(명령어):
주어진 좌표를 기준으로 가장 가까운 공항을 구하십시오.
40.728157, -73.794853

### Response(응답):입력된 좌표 기준 가장 가까운 공항은 40.728157과 73.7944853의 중간 지점인 미국 하와이 호놀룰루입니다.

### Instruction(명령어):
동물의 서식지를 시뮬레이션하는 프로그램을 설계하십시오.

### Response(응답):동물의 서식지 시뮬레이션 프로그램을 설계하기 위해서는, 해당 지역의 환경과 생태계, 생물 다양성 등을 고려해야 합니다. 이를 위해 데이터 수집, 모델링 및 시뮬레이션을 수행할 수 있는 알고리즘이 필요합니다.


In [33]:
generator = pipeline('text-generation', model='/content/drive/MyDrive/PPO/PPO_result', tokenizer=tokenizer)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [ dataset['prompt'][30], dataset['prompt'][500] ,dataset['prompt'][1100], dataset['prompt'][2000]]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **beam_generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
식물의 세포 호흡 과정을 설명하십시오.

### Response(응답):식물은 광합성을 통해 이산화탄소와 물을 흡수하고 산소를 방출하여 에너지를 생산합니다.

### Instruction(명령어):
로미오와 줄리엣에 나오는 로미오의 성격을 묘사하십시오.

### Response(응답):로미오는 강하고 용감하며, 로맨틱한 성격입니다.

### Instruction(명령어):
주어진 좌표를 기준으로 가장 가까운 공항을 구하십시오.
40.728157, -73.794853

### Response(응답):입력된 좌표 기준의 가장 가까운 공항은 40.728157과 73.7944853 사이에 위치한 미국 하와이 호놀룰루입니다.

### Instruction(명령어):
동물의 서식지를 시뮬레이션하는 프로그램을 설계하십시오.

### Response(응답):동물 서식지 시뮬레이션 프로그램을 설계하기 위해서는 동물의 종류, 크기, 지리적 위치, 생태계 및 생물 다양성 등 다양한 요소를 고려해야 합니다. 이러한 요소들을 고려하여 실제 환경과 유사한 환경을 구축하고, 각 동물에 대한 데이터를 수집하고 이를 분석하여 모델링할 수 있는 알고리즘을 개발해야 합니다.


In [30]:
generator = pipeline('text-generation', model='/content/drive/MyDrive/merged_model', tokenizer=tokenizer)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [ dataset['prompt'][30], dataset['prompt'][500] ,dataset['prompt'][1100], dataset['prompt'][2000]]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **greedy_generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



### Instruction(명령어):
식물의 세포 호흡 과정을 설명하십시오.

### Response(응답)::식물은 광합성을 통해 이산화탄소와 물을 흡수합니다. 이산화탄소는 산소와 결합하여 산소를 생성하고, 물은 이산화탄소와 물로 분해됩니다.

### Instruction(명령어):
로미오와 줄리엣에 나오는 로미오의 성격을 묘사하십시오.

### Response(응답)::로미오는 사랑스럽고 용감하며 로맨틱한 성격을 가지고 있습니다.

### Instruction(명령어):
주어진 좌표를 기준으로 가장 가까운 공항을 구하십시오.
40.728157, -73.794853

### Response(응답):40.728157, -73.794853의 가장 가까운 공항은 Airport Center입니다.

### Instruction(명령어):
동물의 서식지를 시뮬레이션하는 프로그램을 설계하십시오.

### Response(응답)::동물의 서식지를 시뮬레이션하는 프로그램을 설계하는 방법은, 먼저, 동물들의 서식지를 조사하고, 그 후 각 동물들의 서식지를 기반으로 시뮬레이션을 수행합니다.


In [32]:
generator = pipeline('text-generation', model='/content/drive/MyDrive/PPO/PPO_result', tokenizer=tokenizer)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [ dataset['prompt'][30], dataset['prompt'][500] ,dataset['prompt'][1100], dataset['prompt'][2000]]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **greedy_generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



### Instruction(명령어):
식물의 세포 호흡 과정을 설명하십시오.

### Response(응답)::세포 호흡은 세포가 에너지를 생성하기 위해 산소를 사용하는 과정입니다. 이 과정에서 세포는 산소를 흡수하고 이산화탄소를 방출합니다. 이 과정에서 세포는 산소를 흡수하고 이산화탄소를 방출합니다.

### Instruction(명령어):
로미오와 줄리엣에 나오는 로미오의 성격을 묘사하십시오.

### Response(응답):은근하고 친절하며 로맨틱한 성격입니다.

### Instruction(명령어):
주어진 좌표를 기준으로 가장 가까운 공항을 구하십시오.
40.728157, -73.794853

### Response(응답):40.728157, -73.794853의 거리는 약 1,092km입니다.

### Instruction(명령어):
동물의 서식지를 시뮬레이션하는 프로그램을 설계하십시오.

### Response(응답)::동물의 서식지를 시뮬레이션하는 프로그램을 설계하는 것은 매우 복잡합니다. 예를 들어, 어떤 동물들이 서식하는지, 어떤 종류의 동물이 살고 있는지, 그리고 어떤 종류의 동물이 가장 많이 서식하는지에 대한 정보를 수집해야 합니다. 이러한 정보를 기반으로 시뮬레이션 프로그램을 설계하고 실행할 수 있습니다.


In [36]:
import random

generator1 = pipeline('text-generation', model='/content/drive/MyDrive/merged_model', tokenizer=tokenizer)
generator2 = pipeline('text-generation', model='/content/drive/MyDrive/PPO/PPO_result', tokenizer=tokenizer)

In [42]:
torch.cuda.empty_cache()

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = random.sample(dataset['prompt'], 4)

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator1(list_prompt, **beam_generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))
print('\n', 'which one is better?')
list_result = generator2(list_prompt, **beam_generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))


### Instruction(명령어):
전통적인 비즈니스 형태와 현대적인 비즈니스 방식을 비교하고 대조합니다.

### Response(응답):전통적인 비즈니스와 현대적인 비즈니스의 차이점은 다음과 같습니다: 전통 비즈니스는 고객 서비스, 제품 및 서비스 제공에 중점을 둡니다. 반면, 현대적인 비즈니스는 창의성과 혁신에 중점을 둔다.

### Instruction(명령어):
React Native와 Flutter의 기능을 비교하고 대조합니다.

### Response(응답):Flutter는 자동화된 프로그래밍 언어이며, React Natives는 대화형 프로그래밍 언어입니다.

### Instruction(명령어):
다음 맛을 설명하는 고유 단어 다섯 개를 생성하세요.
Sour

### Response(응답):달콤한, 신맛, 쓴맛, 짠맛

### Instruction(명령어):
기후 변화에 대처하기 위한 계획을 작성하세요.

### Response(응답):기후 변화에 대응하기 위한 계획은 다음과 같습니다: 에너지 절약, 친환경 교통수단 사용, 재활용 및 재사용 촉진 등입니다.

 which one is better?

### Instruction(명령어):
전통적인 비즈니스 형태와 현대적인 비즈니스 방식을 비교하고 대조합니다.

### Response(응답):전통적인 비즈니스와 현대적인 비즈니스의 차이점은 다음과 같습니다: 전통 비즈니스는 대규모 자본을 필요로 하지만, 현대적인 비즈니스는 소규모 자본으로도 창업이 가능합니다. 또한 전통 비즈니스는 더 높은 수익을 창출할 수 있지만, 현대적인 비즈니스에서는 더 적은 비용으로 더 많은 고객을 유치할 수 있습니다.

### Instruction(명령어):
React Native와 Flutter의 기능을 비교하고 대조합니다.

### Response(응답):Flutter는 더 직관적인 인터페이스를 제공하지만, React Natives는 좀 더 개인화된 경험을 제공합니다.

### Instruction(명령어):
다음

정성 평가 결과, PPO가 약 7:3 정도의 비율로 SFT보다 더 만족스러운 응답을 생성하는 것으로 판단되었으며, 두 모델의 응답이 유사한 경우도 상당수 확인되었다. 다만 외부 데이터 자체의 오류나 부정확한 프롬프트가 포함된 경우에는 두 모델 모두 유사한 유형의 오답을 내는 경향이 있었다.

PPO 모델의 추가 개선 방향으로는 엔트로피 증가를 제어하여 불필요한 탐색을 억제하는 전략이 고려될 수 있다. 또한 Reward Model의 설계 측면에서도, 현재는 마지막 유효 토큰 기준으로 reward를 계산하고 있으나, 이를 전체 유효 토큰에 대한 평균 혹은 다른 방식으로 확장함으로써 reward의 안정성을 높일 여지가 있다.

궁극적으로 PPO의 성능 개선은 RM의 품질에 의해 결정되며, 현재 구성된 RM 학습 데이터셋은 SFT에 사용되지 않은 prompt에 대해 gold, beam, greedy 응답을 비교하여 구성되어 있어, 추가적인 성능 향상을 유도하기 어려운 한계가 있다.

따라서 향후 개선 방향으로는 RM 데이터셋을 보다 정교하게 구성하는 것이 필요하며, 이를 위해 외부의 우수한 모델 또는 웹 검색 등의 외부 지식 접근이 가능한 모델을 활용하여 보다 신뢰도 높은 응답을 확보하고, 이를 reward 기준으로 사용하는 방식이 효과적일 것으로 기대된다.

trial

In [195]:
from torch.nn.utils.rnn import pad_sequence

for batch in trainer.dataloader:
  query_tensors = batch["input_ids"]

  with torch.no_grad():
    response_tensors = actor_model.generate(query_tensors,attention_mask=batch['attention_mask'], **generation_kwargs)

  response_list=tokenizer.batch_decode(response_tensors, skip_special_tokens=True)
  rm_tokens = [torch.tensor(tokenizer.encode(s)) for s in response_list]
  rm_tokens_pad = pad_sequence(rm_tokens, batch_first=True, padding_value=tokenizer.pad_token_id)
  rm_attention_mask = (rm_tokens_pad != tokenizer.pad_token_id).long()

  rm_model.to(device)
  with torch.no_grad():
    rewards = rm_model(rm_tokens_pad.to(device), rm_attention_mask.to(device))
  rm_model.to("cpu")
  del rm_tokens, rm_tokens_pad, rm_attention_mask
  torch.cuda.empty_cache()

  # for i in len(rewards):
  #   query = query_tensors[i]
  #   reward = rewards[i]
  #   response = response_tensors[i]
  #   real_response = response[query_tensors.shape[1]:]

  real_response = response_tensors[:, query_tensors.shape[1]:]

  stats = trainer.batched_step(query_tensors, real_response , rewards)

  print(f"[Step 1] KL={stats['objective/kl']:.4f}, Loss={stats['objective/loss']:.4f}")
  del query_tensors, response_tensors, real_response, rewards
  torch.cuda.empty_cache()
  break

AttributeError: 'PPOTrainer' object has no attribute 'batched_step'

In [ ]:
from trl.core import logprobs_from_logits

def compute_reward(prompt_input_ids, response_input_ids, model, ref_model, reward_model):
  full_input = torch.cat([prompt_input_ids, response_input_ids], dim=1)
  model_outputs = model(full_input)
  ref_outputs = ref_model(full_input)

  logprobs = logprobs_from_logits(model_outputs.logits, full_input)
  ref_logprobs = logprobs_from_logits(ref_outputs.logits, full_input)
  kl = logprobs - ref_logprobs

  score = reward_model(full_input)
  reward = -kl.mean(dim=-1) + score
  return reward

In [ ]:
from math import ceil

n_data = len(dataset)
min_steps = ceil(n_data / ppo_config.batch_size)

In [ ]:
# model init
    "sft_model_path": "/content/drive/MyDrive/merged_model",

In [ ]:
class CriticWithScore(AutoModelForCausalLMWithValueHead):
  def score(self, hidden_states):
    return self.v_head(hidden_states).squeeze(-1)
  def forward(self, *args, **kwargs):
        kwargs.setdefault("output_hidden_states", True)
        kwargs.setdefault("return_dict", True)
        return super().forward(*args, **kwargs)

class CriticWithScore(AutoModelForCausalLMWithValueHead):
    def score(self, hidden_states):
        return self.v_head(hidden_states).squeeze(-1)

    def forward(self, *args, **kwargs):
        kwargs.setdefault("output_hidden_states", True)
        kwargs.setdefault("return_dict", True)
        outputs = super().forward(*args, **kwargs)

        # hidden_states는 transformer의 결과에만 있음
        # outputs.hidden_states가 없으면 여기서 직접 전달
        if not hasattr(outputs, 'hidden_states'):
            # transformer에 직접 접근해서 hidden_states 추출
            transformer_outputs = self.transformer(
                *args,
                **kwargs,
            )
            hidden_states = transformer_outputs.hidden_states
            outputs = CausalLMOutputWithCrossAttentions(
                loss=outputs.loss if hasattr(outputs, "loss") else None,
                logits=outputs.logits,
                past_key_values=outputs.past_key_values if hasattr(outputs, "past_key_values") else None,
                hidden_states=hidden_states,
                attentions=outputs.attentions if hasattr(outputs, "attentions") else None,
                cross_attentions=None,
            )

        return outputs

In [ ]:
print("Inside trainer:")
print("reward_model =", type(trainer.reward_model))
print("value_model =", type(trainer.value_model))
print("Has trainer.reward_model.score?", hasattr(trainer.reward_model, "score"))
print("Has trainer.value_model.score?", hasattr(trainer.value_model, "score"))

In [42]:
for batch in trainer.dataloader:
    print(batch)
    print(type(batch))
    break

{'input_ids': tensor([[30132, 42872, 33313,  ...,     3,     3,     3],
        [30132, 42872, 33313,  ...,     3,     3,     3],
        [30132, 42872, 33313,  ...,     3,     3,     3],
        ...,
        [30132, 42872, 33313,  ...,     3,     3,     3],
        [30132, 42872, 33313,  ...,     3,     3,     3],
        [30132, 42872, 33313,  ...,     3,     3,     3]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [ ]:
tokenizer.padding_side = "left"  # generation 시 필수
actor_model.config.pad_token_id
tokenizer.encode("<pad>")
tokenizer.decode([8])